In [74]:
import pandas as pd
import numpy as np 

from itertools import combinations

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [75]:
paths = {
    "ADNIMERGE": "data/ADNIMERGE_17Apr2025.csv",
    "APOERES": "data/APOERES_17Apr2025.csv",
    "CDR": "data/CDR_24Apr2025.csv",
    "FAQ": "data/FAQ_24Apr2025.csv",
    "NEUROBAT": "data/NEUROBAT_17Apr2025.csv",
    "DEMOG": "data/RMT_PTDEMOG_17Apr2025.csv",
    "PSYCH": "data/UWNPSYCHSUM_17Apr2025.csv"
}

dfs = {name: pd.read_csv(path) for name, path in paths.items()}

df_adni = dfs["ADNIMERGE"]

C:\Users\est.pedrogs\AppData\Local\Temp\ipykernel_23956\4170551003.py:11: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = {name: pd.read_csv(path) for name, path in paths.items()}
C:\Users\est.pedrogs\AppData\Local\Temp\ipykernel_23956\4170551003.py:11: DtypeWarning: Columns (79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = {name: pd.read_csv(path) for name, path in paths.items()}


In [76]:
# Filter each dataset to include only baseline records (VISCODE == 'bl'),
# ensuring consistency by using only the first visit per subject across all files.
# This avoids mixing longitudinal follow-up data and keeps the target (DX_bl) aligned.
for name in dfs:
    if 'VISCODE' in dfs[name].columns:
        dfs[name] = dfs[name][dfs[name]['VISCODE'] == 'bl']

In [77]:
for name, df in dfs.items():
    has_rid = 'RID' in df.columns
    print(f"{name}: {'has RID' if has_rid else 'no RID'}")


ADNIMERGE: has RID
APOERES: has RID
CDR: has RID
FAQ: has RID
NEUROBAT: has RID
DEMOG: no RID
PSYCH: has RID


In [78]:
print(dfs['DEMOG'].columns)

Index(['RMT_PHASE', 'ADNIOnlineID', 'PTID', 'RMT_Timepoint', 'RMT_StatusDate',
       'Age_Baseline', 'Gender', 'LatinoEthnicity', 'Latino_MX', 'Latino_PR',
       'Latino_CB', 'Latino_Other', 'Race_AmerIndian', 'Race_Asian',
       'Race_AfricanAmerican', 'Race_PacificIslander', 'Race_White',
       'Race_Unknown', 'Race_PreferNotSay', 'RMT_Education',
       'RMT_ADI_NATRANK_v2021', 'RMT_ADI_STATERANK_v2021'],
      dtype='object')


In [79]:
# Remove feature DEMOG because it has no RID for merge
_ = dfs.pop('DEMOG', None)

In [80]:
dfs.keys()

dict_keys(['ADNIMERGE', 'APOERES', 'CDR', 'FAQ', 'NEUROBAT', 'PSYCH'])

In [81]:
# Start merging all datasets using 'RID' as the key.
# ADNIMERGE is used as the base since it contains the diagnosis labels (DX_bl).
# Perform a left join to retain all records from ADNIMERGE, even if some datasets lack entries for a given RID.

df_merged = dfs['ADNIMERGE']

for df in dfs:
    if df != 'ADNIMERGE':     
        df_merged = df_merged.merge(dfs[df], on='RID', how='left', suffixes=('', f'_{df.lower()}'))

# Drop duplicate columns created during merging to ensure feature uniqueness
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]
# Keep only rows that have a known baseline diagnosis (DX_bl), required for supervised classification
df = df_merged.dropna(subset=['DX_bl'])


In [82]:
df['DX_bl'].value_counts()

DX_bl
LMCI    690
CN      542
EMCI    423
AD      411
SMC     353
Name: count, dtype: int64

In [83]:
print("Original ADNIMERGE shape (all visits, unfiltered):", df_adni.shape)
print("Merged dataset shape (baseline only, before dropping missing targets):", df_merged.shape)
print("Final dataset shape (baseline only, with non-null diagnosis):", df.shape)

Original ADNIMERGE shape (all visits, unfiltered): (16421, 116)
Merged dataset shape (baseline only, before dropping missing targets): (2430, 273)
Final dataset shape (baseline only, with non-null diagnosis): (2419, 273)


In [84]:
# Drop columns with more than 40% missing values
threshold = 0.55
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > threshold].index
df_clean = df.drop(columns=cols_to_drop)
df_clean.shape

(2419, 190)

In [85]:
cols_duplicated = {}
cols = df_clean.columns.tolist()
for x, y in combinations(cols, 2):
    df_x = df_clean[x]
    df_y = df_clean[y]
    
    n = 0
    for a, b in zip(df_x, df_y):
        if a == b or (pd.isna(a) and pd.isna(b)):
            n += 1
    if len(df_x)*0.9 < n:        
        cols_duplicated[x,y] = n

In [86]:
# sort_cols_duplicated = sorted(cols_duplicated.items())
# sort_cols_duplicated

sort_cols_duplicated = sorted(cols_duplicated.items(), key=lambda item: item[1], reverse=True)
sort_cols_duplicated, len(sort_cols_duplicated)

([(('COLPROT', 'ORIGPROT'), 2419),
  (('EXAMDATE', 'EXAMDATE_bl'), 2419),
  (('FDG', 'FDG_bl'), 2419),
  (('AV45', 'AV45_bl'), 2419),
  (('ABETA', 'ABETA_bl'), 2419),
  (('TAU', 'TAU_bl'), 2419),
  (('PTAU', 'PTAU_bl'), 2419),
  (('CDRSB', 'CDRSB_bl'), 2419),
  (('ADAS11', 'ADAS11_bl'), 2419),
  (('ADAS13', 'ADAS13_bl'), 2419),
  (('ADASQ4', 'ADASQ4_bl'), 2419),
  (('MMSE', 'MMSE_bl'), 2419),
  (('RAVLT_immediate', 'RAVLT_immediate_bl'), 2419),
  (('RAVLT_learning', 'RAVLT_learning_bl'), 2419),
  (('RAVLT_forgetting', 'RAVLT_forgetting_bl'), 2419),
  (('RAVLT_perc_forgetting', 'RAVLT_perc_forgetting_bl'), 2419),
  (('LDELTOTAL', 'LDELTOTAL_BL'), 2419),
  (('TRABSCOR', 'TRABSCOR_bl'), 2419),
  (('FAQ', 'FAQ_bl'), 2419),
  (('MOCA', 'MOCA_bl'), 2419),
  (('EcogPtMem', 'EcogPtMem_bl'), 2419),
  (('EcogPtLang', 'EcogPtLang_bl'), 2419),
  (('EcogPtVisspat', 'EcogPtVisspat_bl'), 2419),
  (('EcogPtPlan', 'EcogPtPlan_bl'), 2419),
  (('EcogPtOrgan', 'EcogPtOrgan_bl'), 2419),
  (('EcogPtDivatt',

In [87]:
pd.options.display.max_columns = None
df_clean

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,AV45,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,IMAGEUID_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp,PHASE_faq,PTID_faq,VISCODE_faq,VISCODE2_faq,VISDATE_faq,SOURCE,FAQFINAN,FAQFORM,FAQSHOP,FAQGAME,FAQBEVG,FAQMEAL,FAQEVENT,FAQTV,FAQREM,FAQTRAVL,FAQTOTAL,ID_faq,SITEID_faq,USERDATE_faq,update_stamp_faq,PHASE_neurobat,PTID_neurobat,VISCODE_neurobat,VISCODE2_neurobat,VISDATE_neurobat,CLOCKCIRC,CLOCKSYM,CLOCKNUM,CLOCKHAND,CLOCKTIME,CLOCKSCOR,COPYCIRC,COPYSYM,COPYNUM,COPYHAND,COPYTIME,COPYSCOR,AVTOT1,AVERR1,AVTOT2,AVERR2,AVTOT3,AVERR3,AVTOT4,AVERR4,AVTOT5,AVERR5,AVTOT6,AVERR6,AVTOTB,AVERRB,CATANIMSC,CATANPERS,CATANINTR,TRAASCOR,TRAAERRCOM,TRAAERROM,TRABSCOR_neurobat,TRABERRCOM,TRABERROM,AVDEL30MIN,AVDELERR1,AVDELTOT,AVDELERR2,ANARTERR,ID_neurobat,SITEID_neurobat,USERDATE_neurobat,update_stamp_neurobat,PHASE_psych,VISCODE_psych,VISCODE2_psych,EXAMDATE_psych,ADNI_MEM,ADNI_EF,ADNI_LAN,ADNI_VS,ADNI_EF2,update_stamp_psych
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.33615,NaN,NaN,NaN,NaN,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,112.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,CN,-4.310280,-4.114430,2005-09-08,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,112.0,0.0,-4.310280,-4.114430,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33615,NaN,0.0,0.0,0,0,2023-07-07 04:59:40.0,ADNI1,011_S_0002,bl,bl,2005-09-08,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,107.0,2005-09-08,2005-09-08 00:00:00.0,ADNI1,011_S_0002,bl,bl,2005-09-08,1.0,0.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,7.0,0.0,7.0,0.0,9.0,0.0,10.0,0.0,11.0,0.0,5.0,2.0,4.0,1.0,15.0,2.0,0.0,29.0,0.0,0.0,112.0,1.0,0.0,5.0,4.0,6.0,0.0,16.0,30.0,107.0,2005-09-09,2014-08-26 16:22:44.0,ADNI1,bl,bl,2005-09-08,0.405,-0.041,0.140,0.739,-0.310,2023-12-11 14:53:52.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.10860,NaN,741.5,239.7,22.83,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,148.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,-16.244900,-15.829900,2005-09-12,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,148.0,10.0,-16.244900,-15.829900,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.10860,NaN,0.0,0.0,0,0,2023-07-07 04:59:40.0,ADNI1,011_S_0003,bl,bl,2005-09-12,1.0,2.0,3.0,2.0,3.0,1.0,1.0,4.0,3

In [88]:
drop_cols = [
    "EXAMDATE",
    "PHASE_faq",
    "PTID_faq",
    "RID",
    "USERDATE_faq",
    "USERDATE_neurobat",
    "VISCODE",
    "VISCODE_faq",
    "VISDATE_faq",
    "update_stamp",
    "update_stamp_faq",
    "update_stamp_neurobat",
    "update_stamp_psych",
]
others_Cols = []
for (x, y), n in sort_cols_duplicated:
    if not y in drop_cols:
        drop_cols.append(y)

df = df_clean.drop(columns=drop_cols)
df


,COLPROT,PTID,SITE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,AV45,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,Years_bl,SOURCE,FAQFINAN,FAQFORM,FAQSHOP,FAQGAME,FAQBEVG,FAQMEAL,FAQEVENT,FAQTV,FAQREM,FAQTRAVL,FAQTOTAL,ID_faq,SITEID_faq,CLOCKCIRC,CLOCKSYM,CLOCKTIME,CLOCKSCOR,COPYSCOR,AVTOT1,AVERR1,AVTOT2,AVERR2,AVTOT3,AVERR3,AVTOT4,AVERR4,AVTOT5,AVERR5,AVTOT6,AVERR6,AVTOTB,AVERRB,CATANIMSC,CATANPERS,TRAASCOR,TRAAERRCOM,TRABSCOR_neurobat,TRABERRCOM,AVDEL30MIN,AVDELERR1,AVDELTOT,AVDELERR2,ANARTERR,ID_neurobat,ADNI_MEM,ADNI_EF,ADNI_LAN,ADNI_VS,ADNI_EF2
0,ADNI1,011_S_0002,11,CN,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.33615,NaN,NaN,NaN,NaN,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,112.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,CN,-4.310280,-4.114430,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,107.0,1.0,0.0,1.0,4.0,5.0,7.0,0.0,7.0,0.0,9.0,0.0,10.0,0.0,11.0,0.0,5.0,2.0,4.0,1.0,15.0,2.0,29.0,0.0,112.0,1.0,5.0,4.0,6.0,0.0,16.0,30.0,0.405,-0.041,0.140,0.739,-0.310
1,ADNI1,011_S_0003,11,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.10860,NaN,741.5,239.7,22.83,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,148.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,-16.244900,-15.829900,0.0,1.0,2.0,3.0,2.0,3.0,1.0,1.0,4.0,3.0,3.0,4.0,10.0,8.0,107.0,1.0,0.0,0.0,2.0,4.0,3.0,0.0,6.0,0.0,4.0,0.0,5.0,0.0,4.0,0.0,3.0,1.0,1.0,2.0,10.0,0.0,57.0,0.0,148.0,1.0,0.0,0.0,1.0,0.0,11.0,36.0,-1.021,-1.144,-1.368,-0.880,-1.062
2,ADNI1,022_S_0004,22,LMCI,67.5,Male,10,Hisp/Latino,White,Married,0.0,NaN,NaN,1501.0,153.1,13.29,1.0,14.33,21.33,6.0,27.0,37.0,7.0,4.0,36.3636,4.0,271.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,64631.0,39605.0,6869.0,1154980.0,3983.0,19036.0,19615.0,1679440.0,MCI,-8.421500,-9.449350,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,10.0,1.0,0.0,1.0,4.0,5.0,4.0,0.0,6.0,1.0,8.0,1.0,8.0,1.0,11.0,0.0,6.0,0.0,2.0,1.0,22.0,0.0,67.0,1.0,271.0,3.0,7.0,6.0,13.0,1.0,NaN,140.0,-0.009,-0.934,0.339,-0.739,-1.181
3,ADNI1,011_S_0005,11,CN,73.7,Male,16,Not Hisp/Latino,White,Married,0.0,1.25956,NaN,547.3,337.0,33.43,0.0,8.67,14.67,4.0,29.0,37.0,4.0,4.0,44.4444,12.0,90.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32246.0,34062.0,7075.0,1116630.0,4433.0,24788.0,21614.0,1640770.0,CN,-1.888680,-1.565830,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,107.0,1.0,1.0,1.0,5.0,5.0,5.0,1.0,6.0,1.0,9.0,0.0,8.0,0.0,9.0,0.0,4.0,0.0,4.0,1.0,19.0,1.0,50.0,0.0,90.0,1.0,5.0,5.0,15.0,1.0,5.0,32.0,0.743,0.165,0.259,0.739,-0.152
4,ADNI1,100_S_0006,100,LMCI,80.4,Female,13,Not Hisp/Latino,White,Married,0.0,NaN,NaN,NaN,NaN,NaN,0.5,18.67,25.67,7.0,25.0,30.0,1.0,5.0,83.3333,3.0,168.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,33025.0,39826.0,5348.0,927510.0,2277.0,17963.0,17802.0,1485830.0,MCI,-10.141700,-10.910900,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,44.0,4.0,1.0,1.0,0.0,4.0,4.0,5.0,0.0,6.0,0.0,6.0,0.0,7.0,1.0,6.0,0.0,4.0,4.0,4.0,0.0,13.0,4.0,49.0,0.0,168.0,1.0,1.0,0.0,7.0,1.0,17.0,182.0,-0.219,-0.463,-0.260,-0.880,-0.498
...,...,...,...,...,...,...,...,.

In [89]:
non_numeric_features = df.select_dtypes(exclude=['number']).columns
print("Non-numeric features:", non_numeric_features)

Non-numeric features: Index(['COLPROT', 'PTID', 'DX_bl', 'PTGENDER', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'ABETA', 'TAU', 'PTAU', 'FLDSTRENG', 'FSVERSION', 'DX'],
      dtype='object')


In [90]:
df_non_numeric = df[non_numeric_features]
df_non_numeric.head()

,COLPROT,PTID,DX_bl,PTGENDER,PTETHCAT,PTRACCAT,PTMARRY,ABETA,TAU,PTAU,FLDSTRENG,FSVERSION,DX
0,ADNI1,011_S_0002,CN,Male,Not Hisp/Latino,White,Married,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,CN
1,ADNI1,011_S_0003,AD,Male,Not Hisp/Latino,White,Married,741.5,239.7,22.83,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,Dementia
2,ADNI1,022_S_0004,LMCI,Male,Hisp/Latino,White,Married,1501.0,153.1,13.29,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,MCI
3,ADNI1,011_S_0005,CN,Male,Not Hisp/Latino,White,Married,547.3,337.0,33.43,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,CN
4,ADNI1,100_S_0006,LMCI,Female,Not Hisp/Latino,White,Married,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,MCI


In [91]:
label_encoders = {}
for col in df_non_numeric.columns:
    df[col] = df[col].astype(str)  # Ensure all values are strings
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store the encoder in case we need inverse_transform later

print(f"Encoded {len(df_non_numeric.columns)} non-numeric features using LabelEncoder.")

Encoded 13 non-numeric features using LabelEncoder.


In [92]:
# Reset the display option to show a limited number of columns again
pd.reset_option('display.max_columns')
df = df.drop('PTID', axis=1)
display(df)

,COLPROT,SITE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,...,AVDELERR1,AVDELTOT,AVDELERR2,ANARTERR,ID_neurobat,ADNI_MEM,ADNI_EF,ADNI_LAN,ADNI_VS,ADNI_EF2
0,0,11,1,74.3,1,16,1,6,1,0.0,...,4.0,6.0,0.0,16.0,30.0,0.405,-0.041,0.140,0.739,-0.310
1,0,11,0,81.3,1,18,1,6,1,1.0,...,0.0,1.0,0.0,11.0,36.0,-1.021,-1.144,-1.368,-0.880,-1.062
2,0,22,3,67.5,1,10,0,6,1,0.0,...,6.0,13.0,1.0,NaN,140.0,-0.009,-0.934,0.339,-0.739,-1.181
3,0,11,1,73.7,1,16,1,6,1,0.0,...,5.0,15.0,1.0,5.0,32.0,0.743,0.165,0.259,0.739,-0.152
4,0,100,3,80.4,0,13,1,6,1,0.0,...,0.0,7.0,1.0,17.0,182.0,-0.219,-0.463,-0.260,-0.880,-0.498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,2,33,4,65.6,0,17,1,2,0,NaN,...,0.0,15.0,0.0,5.0,170226.0,2.978,2.087,0.960,0.739,2.026
2425,2,53,3,61.4,1,12,1,6,1,NaN,...,2.0,8.0,6.0,11.0,170105.0,-0.844,-1.490,-0.851,-0.080,-1.596
2426,2,35,3,64.6,0,12,0,5,4,NaN,...,1.0,11.0,1.0,12.0,171669.0,-0.232,0.310,0.606,-0.739,0.160
2427,2,82,2,68.9,0,18,1,2,2,NaN,...,1.0,13.0,1.0,9.0,173872.0,1.392,1.313,0.856,0.739,1.056


In [95]:
print(df.info())
print(df.value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 2419 entries, 0 to 2428
Columns: 106 entries, COLPROT to ADNI_EF2
dtypes: float64(92), int32(12), int64(2)
memory usage: 1.9 MB
None
COLPROT  SITE  DX_bl  AGE   PTGENDER  PTEDUCAT  PTETHCAT  PTRACCAT  PTMARRY  APOE4  FDG       AV45    ABETA  TAU   PTAU  CDRSB  ADAS11  ADAS13  ADASQ4  MMSE  RAVLT_immediate  RAVLT_learning  RAVLT_forgetting  RAVLT_perc_forgetting  LDELTOTAL  TRABSCOR  FAQ   MOCA  EcogPtMem  EcogPtLang  EcogPtVisspat  EcogPtPlan  EcogPtOrgan  EcogPtDivatt  EcogPtTotal  EcogSPMem  EcogSPLang  EcogSPVisspat  EcogSPPlan  EcogSPOrgan  EcogSPDivatt  EcogSPTotal  FLDSTRENG  FSVERSION  IMAGEUID   Ventricles  Hippocampus  WholeBrain  Entorhinal  Fusiform  MidTemp  ICV        DX  mPACCdigit  mPACCtrailsB  Years_bl  SOURCE  FAQFINAN  FAQFORM  FAQSHOP  FAQGAME  FAQBEVG  FAQMEAL  FAQEVENT  FAQTV  FAQREM  FAQTRAVL  FAQTOTAL  ID_faq    SITEID_faq  CLOCKCIRC  CLOCKSYM  CLOCKTIME  CLOCKSCOR  COPYSCOR  AVTOT1  AVERR1  AVTOT2  AVERR2  AVTOT3  AV